In [1]:
import pandas as  pd
import numpy as np
import warnings
import itertools
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm
from numpy import loadtxt
import time
import gc
warnings.filterwarnings('ignore')
kernel_with_output = True
np.random.seed(10)
%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 231)
pd.set_option('display.max_columns', 100)

In [3]:
Validation = False
start_time = time.time()

In [4]:
if kernel_with_output:
    # Feature engineering list
    new_features = []
    enable_feature_idea = [True, True, True, True, True, True, True, True, True, True]
    # Some parameters(maybe add more periods, score will be better) [1,2,3,12]
    lookback_range = [1,2,3,4,5,6,7,8,9,10,11,12]
    tqdm.pandas()

In [5]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)
    return df

In [7]:
if kernel_with_output:
    item =pd.read_csv("../data/items.csv")
    item_cat = pd.read_csv("../data/item_categories.csv")
#     df_shops = pd.read_csv("../data/shops.csv")
    sale_train = pd.read_csv("../data/sales_train.csv")
    test = pd.read_csv("../data/test.csv")
    temp = pd.read_csv("../data/sample_submission.csv")


In [8]:
sale_train.loc[2909818] # id = 2909818 seems outliers

date              28.10.2015
date_block_num            33
shop_id                   12
item_id                11373
item_price          0.908714
item_cnt_day            2169
Name: 2909818, dtype: object

In [9]:
sale_train['item_price'][2909818] = sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_price'].median()
sale_train['item_cnt_day'][2909818] = round(sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_cnt_day'].median())

In [10]:
sale_train.loc[2909818] 

date              28.10.2015
date_block_num            33
shop_id                   12
item_id                11373
item_price               317
item_cnt_day               4
Name: 2909818, dtype: object

In [11]:
sale_train.loc[885138] # the price seems too high

date              17.09.2013
date_block_num             8
shop_id                   12
item_id                11365
item_price             59200
item_cnt_day               1
Name: 885138, dtype: object

In [12]:
sale_train['item_price'][885138] = sale_train[(sale_train['item_id'] == 11365) & (sale_train['shop_id'] ==12) & (sale_train['date_block_num'] == 8)]['item_price'].median()

In [13]:
sale_train.loc[885138] # the price seems too high

date              17.09.2013
date_block_num             8
shop_id                   12
item_id                11365
item_price              2770
item_cnt_day               1
Name: 885138, dtype: object

In [14]:
sale_train.shape

(2935849, 6)

In [15]:
test_nrow = test.shape[0]

In [16]:
sale_train = sale_train.merge(test[['shop_id']].drop_duplicates(), how = 'inner') 
sale_train['date'] = pd.to_datetime(sale_train['date'], format = '%d.%m.%Y')

After merge, we delete 522603 records from sale_train. The reason is we want drop all transactions belong to shops in the train set but not in the test. 

In [17]:
sale_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413246 entries, 0 to 2413245
Data columns (total 6 columns):
date              datetime64[ns]
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 128.9 MB


There are 4 steps:

(1) Aggregate data

(2) Add item/shop pair mean-encoding

(3) First-level model

(4) Ensembling

### 1. Aggregate data 

In [ ]:
grid = []
for block_num in sale_train['date_block_num'].unique():
    cur_shops = sale_train[sale_train['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sale_train[sale_train['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(itertools.product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
print('%0.2f min: Finish creating the grid'%((time.time() - start_time)/60))

index_cols = ['shop_id', 'item_id', 'date_block_num']
sale_train['item_cnt_day'] = sale_train['item_cnt_day'].clip(0,20)
# gb_cnt = sale_train.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})
gb_cnt = sale_train.groupby(index_cols).agg({"item_cnt_day":"sum", "item_price":"mean"}).reset_index()
gb_cnt.rename({"item_cnt_day":"item_cnt_month"}, inplace = True, axis = "columns")
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)
#join aggregated data to the grid
train = pd.merge(grid,gb_cnt,how='left',on=index_cols).fillna(0)
train['item_cnt_month'] = train['item_cnt_month'].astype(int)

In [ ]:
train.info()

In [ ]:
train = train.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
test = test.merge(item[["item_id", "item_category_id"]], on = ["item_id"], how="left")

In [ ]:
item_cat.head()
item_cat.info()

In [ ]:
item_cat = pd.read_csv('../data/item_categories.csv')

l_cat = list(item_cat.item_category_name)

for ind in range(0,1):

    l_cat[ind] = 'PC Headsets / Headphones'

for ind in range(1,8):

    l_cat[ind] = 'Access'

l_cat[8] = 'Tickets (figure)'

l_cat[9] = 'Delivery of goods'

for ind in range(10,18):

    l_cat[ind] = 'Consoles'

for ind in range(18,25):

    l_cat[ind] = 'Consoles Games'

l_cat[25] = 'Accessories for games'

for ind in range(26,28):

    l_cat[ind] = 'phone games'

for ind in range(28,32):

    l_cat[ind] = 'CD games'

for ind in range(32,37):

    l_cat[ind] = 'Card'

for ind in range(37,43):

    l_cat[ind] = 'Movie'

for ind in range(43,55):

    l_cat[ind] = 'Books'

for ind in range(55,61):

    l_cat[ind] = 'Music'

for ind in range(61,73):

    l_cat[ind] = 'Gifts'

for ind in range(73,79):

    l_cat[ind] = 'Soft'

for ind in range(79,81):

    l_cat[ind] = 'Office'

for ind in range(81,83):

    l_cat[ind] = 'Clean'

l_cat[83] = 'Elements of a food'


In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
item_cat['item_cat_id_fix'] = lb.fit_transform(l_cat)
train = train.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')
test = test.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')

In [ ]:
item_cat.head(50)

In [ ]:
train = downcast_dtypes(train)
test = downcast_dtypes(test)

In [ ]:
train.info()
test.info()

### 2. Add item/shop pair mean-encodings

##### 2.1 Combine trainset and testset

In [ ]:
print('%0.2f min: Start combining data'%((time.time() - start_time)/60))
if Validation == False:
    test['date_block_num'] = 34
    all_data = pd.concat([train, test], axis = 0)
    all_data = all_data.drop(columns = ['ID'])
else:
    all_data = train

In [ ]:
all_data.shape
all_data.isnull().sum() # test has only 5 cols, so when merge, these cols will be na

In [ ]:
all_data = downcast_dtypes(all_data)

In [ ]:
all_data.head()

In [ ]:
all_data.to_pickle("../results/07/all_data.pkl")

In [ ]:
if kernel_with_output:
    current = time.time()
    trainset = pd.read_pickle("../results/07/all_data.pkl")
    items = pd.read_csv('../data/items.csv')
    shops = pd.read_csv('../data/shops.csv')
    # Only use more recent data
    start_month = 0
    end_month = 33
    trainset = trainset[['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_price', 'item_cnt_month']]
    trainset = trainset[(trainset.date_block_num >= start_month) & (trainset.date_block_num <= end_month)]

    print('Loading test set...')
    test_dataset = loadtxt('../data/test.csv', delimiter="," ,skiprows=1, usecols = (1,2), dtype=int)
    testset = pd.DataFrame(test_dataset, columns = ['shop_id', 'item_id'])

    print('Merging with other datasets...')
    # Get item category id into test_df
    testset = testset.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
    testset['date_block_num'] = 34
    # Make testset contains same column as trainset so we can concatenate them row-wise
    testset['item_cnt_month'] = -1

    train_test_set = pd.concat([trainset, testset], axis = 0) 
    
    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val set')

In [ ]:
train_test_set = downcast_dtypes(train_test_set)

#### Idea 0: Add previous shop/item sales as feature (Lag feature)

In [ ]:
# add the sales of this item of this shop in i previous months, i = loopback_range
if kernel_with_output:
    if enable_feature_idea[0]:
        for diff in tqdm(lookback_range):
            feature_name = 'prev_shopitem_sales_' + str(diff)
            trainset2 = train_test_set.copy()
            trainset2.loc[:, 'date_block_num'] += diff
            trainset2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(trainset2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)
    train_test_set.head(3)

#### Idea 1: Add previous item sales as feature (Lag feature)

In [ ]:
# add the everage of sales of an item in i previous month, ie.
# row item_id = 0, date_block_num = 21, then add a column prev_item_sales_1(the average of sales of item_id = 0 in date_block_num=20) 
if kernel_with_output:
    if enable_feature_idea[1]:
        groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_item_sales_' + str(diff)
            result = groups.agg({'item_cnt_month':'mean'})
            result = result.reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)

In [ ]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/07/all_data_traintest_01.pkl")

#### Idea 2: Add previous shop/item price as feature (Lag feature)

In [ ]:
# add the everage of the price of this item of this shop in i previous months
if kernel_with_output:
    if enable_feature_idea[2]:
        groups = train_test_set.groupby(by = ['item_id', 'shop_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_shopitem_price_' + str(diff)
            result = groups.agg({'item_price':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_price': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'shop_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)

In [ ]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/07/all_data_traintest_012.pkl")

#### Idea 3: Add previous item price as feature (Lag feature)

In [ ]:
# add the everage of the price of this item in i previous months (not care abt shops)
if kernel_with_output:
    if enable_feature_idea[3]:
        groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_item_price_' + str(diff)
            result = groups.agg({'item_price':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_price': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)

In [ ]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/07/all_data_traintest_0123.pkl")

#### Idea 4: mean encoding

In [7]:
train_test_set = pd.read_pickle("../results/07/all_data_traintest_0123.pkl")

In [8]:
def create_mean_encodings(train_test_set, categorical_var_list, target):
    feature_name = "_".join(categorical_var_list) + "_" + target + "_mean"

    df = train_test_set.copy()
    df1 = df[df.date_block_num <= 32]
    df2 = df[df.date_block_num <= 33]
    df3 = df[df.date_block_num == 34]

    # Extract mean encodings using training data(here we don't use month 33 to avoid data leak on validation)
    # If I try to extract mean encodings from all months, then val rmse decreases a tiny bit, but test rmse would increase by 4%
    # So this is important
    mean_32 = df1[categorical_var_list + [target]].groupby(categorical_var_list, as_index=False)[[target]].mean()
    mean_32 = mean_32.rename(columns={target:feature_name})

    # Extract mean encodings using all data, this will be applied to test data
    mean_33 = df2[categorical_var_list + [target]].groupby(categorical_var_list, as_index=False)[[target]].mean()
    mean_33 = mean_33.rename(columns={target:feature_name})

    # Apply mean encodings
    df2 = df2.merge(mean_32, on = categorical_var_list, how = 'left')
    df3 = df3.merge(mean_33, on = categorical_var_list, how = 'left')

    # Concatenate
    train_test_set = pd.concat([df2, df3], axis = 0)
    new_features.append(feature_name)
    return train_test_set


In [10]:
if kernel_with_output:
    train_test_set = create_mean_encodings(train_test_set, ['shop_id', 'item_id'], 'item_cnt_month')
    train_test_set.head(3)

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,prev_shopitem_price_1,prev_shopitem_price_2,prev_shopitem_price_3,prev_shopitem_price_4,prev_shopitem_price_5,prev_shopitem_price_6,prev_shopitem_price_7,prev_shopitem_price_8,prev_shopitem_price_9,prev_shopitem_price_10,prev_shopitem_price_11,prev_shopitem_price_12,prev_item_price_1,prev_item_price_2,prev_item_price_3,prev_item_price_4,prev_item_price_5,prev_item_price_6,prev_item_price_7,prev_item_price_8,prev_item_price_9,prev_item_price_10,prev_item_price_11,prev_item_price_12,shop_id_item_id_item_cnt_month_mean
0,0,37,1.0,22154,999.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909
1,0,40,2.0,22151,399.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
2,0,5,1.0,5603,699.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000


#### Idea 5: Mean encodings for item (Mean encoding, doesnt work for me)

In [11]:
if kernel_with_output:
    train_test_set = create_mean_encodings(train_test_set, ['item_id'], 'item_cnt_month')
    train_test_set.head(3)

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,prev_shopitem_price_1,prev_shopitem_price_2,prev_shopitem_price_3,prev_shopitem_price_4,prev_shopitem_price_5,prev_shopitem_price_6,prev_shopitem_price_7,prev_shopitem_price_8,prev_shopitem_price_9,prev_shopitem_price_10,prev_shopitem_price_11,prev_shopitem_price_12,prev_item_price_1,prev_item_price_2,prev_item_price_3,prev_item_price_4,prev_item_price_5,prev_item_price_6,prev_item_price_7,prev_item_price_8,prev_item_price_9,prev_item_price_10,prev_item_price_11,prev_item_price_12,shop_id_item_id_item_cnt_month_mean,item_id_item_cnt_month_mean
0,0,37,1.0,22154,999.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.104623
1,0,40,2.0,22151,399.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.856000
2,0,5,1.0,5603,699.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.093664


In [12]:
gc.collect()

219

#### Idea 6: Number of month from last sale of shop/item (Use info from past)

In [13]:
# for each item of each shop, find the last i months sales. 
def create_last_sale_shop_item(row):
    for diff in range(1,33+1):
        feature_name = '_prev_shopitem_sales_' + str(diff)
        if row[feature_name] != 0.0:
            return diff
    return np.nan

In [14]:
if kernel_with_output:
    lookback_range = list(range(1, 33 + 1))
    if enable_feature_idea[6]:
        for diff in tqdm(lookback_range):
            feature_name = '_prev_shopitem_sales_' + str(diff)
            trainset2 = train_test_set[["shop_id", "item_id", "date_block_num", "item_cnt_month"]].copy()
            trainset2.loc[:, 'date_block_num'] += diff
            trainset2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(trainset2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            #new_features.append(feature_name)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [06:14<00:00, 13.02s/it]


In [15]:
train_test_set.loc[:, 'last_sale_shop_item'] = train_test_set.progress_apply (lambda row: create_last_sale_shop_item(row),axis=1)

100%|██████████████████████████████████████████████████████████████████████| 8812244/8812244 [21:50<00:00, 6723.13it/s]


In [16]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/07/07_1/all_data_traintest_0123456.pkl")

#### Idea 7: Number of month from last sale of item(Use info from past)

In [17]:
train_test_set_temp = train_test_set[["item_id", "date_block_num", "item_cnt_month"]]

In [18]:
def create_last_sale_item(row):
    for diff in range(1,33+1):
        feature_name = '_prev_item_sales_' + str(diff)
        if row[feature_name] != 0.0:
            return diff
    return np.nan

In [19]:
if kernel_with_output:
    lookback_range = list(range(1, 33 + 1))
    if enable_feature_idea[1]:
        groups = train_test_set_temp.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = '_prev_item_sales_' + str(diff)
            result = groups.agg({'item_cnt_month':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [02:30<00:00,  5.20s/it]


In [20]:
train_test_set.loc[:, 'last_sale_item'] = train_test_set.progress_apply (lambda row: create_last_sale_item(row),axis=1)

100%|█████████████████████████████████████████████████████████████████████| 8812244/8812244 [05:23<00:00, 27274.90it/s]


In [21]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/07/07_1/all_data_traintest_01234567.pkl")

In [11]:
# train_test_set.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1, inplace = True)

In [22]:
train_test_set.loc[train_test_set.prev_item_sales_2=="0*0","prev_item_sales_2"] = 0.0
train_test_set.prev_item_sales_2 = train_test_set.prev_item_sales_2.astype("float32")

In [23]:
if kernel_with_output:
    current = time.time()

    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']

    # Clipping to range 0-20
    train_test_set['item_cnt_month'] = train_test_set.item_cnt_month.fillna(0).clip(0,20)

    # train: want rows with date_block_num from 0 to 31
    train_time_range_lo = (train_test_set['date_block_num'] >= 0)
    train_time_range_hi =  (train_test_set['date_block_num'] <= 32)

    # val: want rows with date_block_num from 22
    validation_time =  (train_test_set['date_block_num'] == 33)

    # test: want rows with date_block_num from 34
    test_time =  (train_test_set['date_block_num'] == 34)


    # Retrieve rows for train set, val set, test set
    cv_trainset = train_test_set[train_time_range_lo & train_time_range_hi]
    cv_valset = train_test_set[validation_time]
    cv_trainset = cv_trainset[baseline_features]
    cv_valset = cv_valset[baseline_features]
    testset = train_test_set[test_time]
    testset = testset[baseline_features]

    # Prepare numpy arrays for training/val/test
    cv_trainset_vals = cv_trainset.values.astype(int)
    trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
    trainy = cv_trainset_vals[:, len(baseline_features) - 1]

    cv_valset_vals = cv_valset.values.astype(int)
    valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
    valy = cv_valset_vals[:, len(baseline_features) - 1]

    testset_vals = testset.values.astype(int)
    testx = testset_vals[:, 0:len(baseline_features) - 1]

    print('Fitting...')
    model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 16)
    model.fit(trainx, trainy, eval_metric='rmse')


    preds = model.predict(valx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)

Fitting...
[15:15:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:15:34] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=11, min_child_weight=0.5, missing=None, n_estimators=100,
       n_jobs=1, nthread=16, num_round=1000, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=1, silent=None, subsample=1, verbosity=1)

In [14]:
    from sklearn.metrics import mean_squared_error
    print('val set rmse: ', np.sqrt(mean_squared_error(valy, preds)))

    preds = model.predict(testx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    df.to_csv('../results/07/07_1.csv')
    print('test predictions written to file')

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val, test set')

val set rmse:  0.9305994642913349
test predictions written to file
Took 986 seconds to train and predict val, test set
